## <b><font color='darkblue'>Preface</font></b>
<font size='3ptx'><b>These seven Pandas tricks will speed up your workflow, cut memory usage, and make your data manipulations smoother. Get ready to level up.</b> [**Pandas**](https://pandas.pydata.org/) is Python's default data-manipulation library. But come on—if you're doing it inefficiently, you're just creating more work than you need to. You ever seen someone iterate over a DataFrame line by line? Torture. <b>Like seeing someone wash a car with a toothbrush</b>.</font>

<font size='3ptx'><b>Pandas is quick, but only if you understand how to use it. The problem is, most don't.</b></font> They use it as a slow, cumbersome spreadsheet instead of the optimized monster that it can be. They use loops when they shouldn't, misuse functions, and then struggle with performance when their datasets grow into tens of thousands of rows.

<b><font size='3ptx'>Here's the reality: Pandas is constructed on top of NumPy, which is optimized for vectorized operations</font></b>. That is to say, wherever possible, <b>you should be operating on whole columns at a time rather than looping over individual rows</b>. Nevertheless, many developers reach for loops instinctively because, well, that's what they're accustomed to. <b>Old habits die hard. But in Pandas, looping is nearly always the slowest way</b>.

<b><font size='3ptx'>Performance isn’t the only problem, though. Code readability matters, too</font></b>. If your Pandas code looks like a tangled mess of `.loc[]`, `.iloc[]`, `.apply()`, and endless conditionals, you’re setting yourself up for frustration both for yourself and anyone else who has to read your work. <b>Clean, efficient Pandas code isn’t just about speed; it’s about writing something that makes sense at a glance</b>.

The best news? <b><font size='3ptx'>Pandas has built-in shortcuts that accelerate, streamline, and make it much less frustrating</font></b>. Some of them are just simple—like applying vectorized operations rather than loops. Some, like `query()` or `merge()`, simply require a small shift in mental thought but save you a tremendous amount of effort. <b>A few tricks will even help minimize memory use, which comes into play when you're working with large sets of data</b>.

<b><font size='3ptx'>These aren't "nice-to-know" hacks. They're the difference between writing Pandas code that works and Pandas code that flies</font></b>. If you're dealing with financial data, scrubbing filthy CSVs, or processing hundreds of thousands of rows, these seven hacks will trim valuable time and suffering from your workflow.

In [31]:
import random
import pandas as pd

test_data = [
    [random.randint(0, 100), random.randint(0, 100)]  for _ in range(50000)
]
test_df = pd.DataFrame(test_data, columns=['a', 'b'])
test_df.sample(n=5)

,a,b
3393,67,64
13002,9,43
25532,66,5
47658,84,12
34522,90,16


## <b><font color='darkblue'>1. Stop Using Loops—Use Vectorized Operations Instead</font></b>

### <b><font color='darkgreen'>The Problem</font></b>
Loops are slow. If you’re iterating through a DataFrame row by row, you’re doing it wrong.

### <b><font color='darkgreen'>Why It Matters</font></b>
Pandas is built on NumPy, which is optimized for fast, vectorized operations. That means instead of looping, you can apply calculations to entire columns at once. It’s faster and less messy.

### <b><font color='darkgreen'>Fix It</font></b>
Instead of this:
```python
df = pd.DataFrame({'a': range(1, 6), 'b': range(10, 15)})
df['c'] = [x * y for x, y in zip(df['a'], df['b'])]
```

Do this:
```python
df['c'] = df['a'] * df['b']
```

Faster, cleaner, and no unnecessary loops.

In [32]:
%%time
test_df['c'] = [x * y for x, y in zip(test_df['a'], test_df['b'])]

CPU times: user 22.7 ms, sys: 24.6 ms, total: 47.4 ms
Wall time: 47.2 ms


In [33]:
%%time
test_df['c'] = test_df['a'] * test_df['b']

CPU times: user 0 ns, sys: 2.04 ms, total: 2.04 ms
Wall time: 1.72 ms


### <b><font color='darkgreen'>Avoid This Mistake</font></b>
`.iterrows()` might seem like a good idea, but it's painfully slow. Use vectorized operations or `.apply()` (but only when needed—see trick #7).

## <b><font color='darkblue'>2. Filter Data Faster with query()</font></b>

### <b><font color='darkgreen'>The Problem</font></b>
Filtering with boolean conditions can get ugly fast.

### <b><font color='darkgreen'>The Fix</font></b>
Instead of:
```python
df[(df['a'] > 2) & (df['b'] < 14)]
```

Use:
```python
df.query('a > 2 and b < 14')
```

More readable, and it runs faster too.

In [36]:
%%time
_ = test_df[(test_df['a'] > 2) & (test_df['b'] < 14)]

CPU times: user 792 μs, sys: 1.28 ms, total: 2.07 ms
Wall time: 2 ms


In [37]:
%%time
_ = test_df.query('a > 2 and b < 14')

CPU times: user 0 ns, sys: 5.56 ms, total: 5.56 ms
Wall time: 4.68 ms


### <b><font color='darkgreen'>Pro Tip</font></b>
If you need to use a variable inside `.query()`, use `@`:

In [39]:
%%time
threshold = 2
_ = test_df.query('a > @threshold')

CPU times: user 2.07 ms, sys: 8.08 ms, total: 10.2 ms
Wall time: 8.99 ms


## <b><font color='darkblue'>3. Save Memory with astype()</font></b>

### <b><font color='darkgreen'>The Problem</font></b>
Large DataFrames eat up RAM.

### <b><font color='darkgreen'>The Fix</font></b>
Downcast data types where possible. Check memory usage before and after with:

In [40]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   a       50000 non-null  int64
 1   b       50000 non-null  int64
 2   c       50000 non-null  int64
dtypes: int64(3)
memory usage: 1.1 MB


In [41]:
test_df['a'] = test_df['a'].astype('int8')

In [42]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   a       50000 non-null  int8 
 1   b       50000 non-null  int64
 2   c       50000 non-null  int64
dtypes: int64(2), int8(1)
memory usage: 830.2 KB


### <b><font color='darkgreen'>Watch Out</font></b>
Downcasting floats can lead to precision loss. Stick to float32 unless you need float64.

## <b><font color='darkblue'>4. Handle Missing Data Without the Headache</font></b>

### <b><font color='darkgreen'>The Problem</font></b>
`NaN` values mess up calculations.

### <b><font color='darkgreen'>The Fix</font></b>
* Remove them: `df.dropna()`
* Fill them: `df.fillna(0)`
* Interpolate them: `df.interpolate()`

### <b><font color='darkgreen'>Pro Tip</font></b>
Interpolation can be a lifesaver for time series data.

## <b><font color='darkblue'>5. Get More From Your Data with groupby()</font></b>

### <b><font color='darkgreen'>The Problem</font></b>
Manually summarizing data is a waste of time.

In [47]:
categories = ["car", "fruit", "others"]
test_data = []
for _ in range(100):
    category = random.choice(categories)
    match category:
        case "car":
            sale = random.randint(5000, 15000)
        case "fruit":
            sale = random.randint(1, 100)
        case _:
            sale = random.randint(1, 10000)
    test_data.append((category, sale))

test_df = pd.DataFrame(test_data, columns = ['category', 'sales'])

### <b><font color='darkgreen'>The Fix</font></b>
Use `groupby()` to aggregate data quickly:

In [48]:
test_df.groupby('category')['sales'].sum()

category
car       360865
fruit       1686
others    140501
Name: sales, dtype: int64

Need multiple aggregations? Use `.agg()`:

In [50]:
test_df.groupby('category').agg({'sales': ['sum', 'mean']})

sales              
             sum          mean
category                      
car       360865  10310.428571
fruit       1686     49.588235
others    140501   4532.290323

### <b><font color='darkgreen'>Did You Know?</font></b>
You can also use `transform()` to add aggregated values back into the original DataFrame without losing the original row structure.

In [51]:
test_df['total_sales'] = test_df.groupby('category')['sales'].transform('sum')

In [53]:
test_df.head(n=10)

,category,sales,total_sales
0,car,9908,360865
1,car,7287,360865
2,car,11886,360865
3,car,13014,360865
4,car,11428,360865
5,fruit,96,1686
6,fruit,41,1686
7,fruit,6,1686
8,fruit,74,1686
9,car,7589,360865


## <b><font color='darkblue'>6. Merge DataFrames Without Slowing Down Your Code</font></b>

### <b><font color='darkgreen'>The Problem</font></b>
Badly executed joins slow everything down.

### <b><font color='darkgreen'>The Fix</font></b>
Use `merge()` properly:

In [54]:
test_data1 = [(1, 'John'), (2, 'Mary'), (3, 'Ken')]
test_data2 = [(1, 45), (2, 23), (3, 37)]
test_df1 = pd.DataFrame(test_data1, columns=['id', 'name']) 
test_df2 = pd.DataFrame(test_data2, columns=['id', 'age']) 

In [56]:
df_merged = test_df1.merge(test_df2, on='id', how='inner')
df_merged

,id,name,age
0,1,John,45
1,2,Mary,23
2,3,Ken,37


### <b><font color='darkgreen'>Best Practice</font></b>
Use `how='left'` if you want to keep all records from the first DataFrame.

### <b><font color='darkgreen'>Performance Tip</font></b>
For large DataFrames, ensure the join key is indexed to speed up merging:
```python
df1.set_index('id', inplace=True)
df2.set_index('id', inplace=True)
df_merged = df1.join(df2, how='inner')
```

## <b><font color='darkblue'>7. Use .apply() the Right Way (and Avoid Overusing It)</font></b>

### <b><font color='darkgreen'>The Problem</font></b>
`.apply()` is powerful but often misused.

### <b><font color='darkgreen'>The Fix</font></b>
Use it for complex row-wise operations:
```python
df['new_col'] = df['a'].apply(lambda x: x**2 if x > 2 else x)
```

But if you're just modifying a single column, use `.map()` instead. It's faster.

### <b><font color='darkgreen'>The Mistake to Avoid</font></b>
Don’t use `.apply()` when a vectorized operation would do the job. `.apply()` is slower than using Pandas' built-in functions.

## <b><font color='darkblue'>Final Thoughts</font></b>
<b><font size='3ptx'>These tricks make your Pandas workflow smoother, faster, and easier to read.</font></b> No more unnecessary loops, no more sluggish joins, just clean, efficient code. Try them out in your next project. If you want to explore them further, check out the [official Pandas documentation](https://pandas.pydata.org/docs/).